In [11]:
import json
import matplotlib.pyplot as plt
import shapely.speedups
from shapely.geometry import Point, Polygon
import numpy as np
import json 
import geopandas as gpd
from mpl_toolkits.basemap import Basemap as Basemap
from matplotlib.colors import rgb2hex
from matplotlib.patches import Polygon
from shapely.ops import unary_union
from shapely.validation import make_valid
shapely.speedups.enable()

# open files to find points within each state
f = open('cities.json')
cities_points = json.load(f)


f = open('states.geojson')
states_geo = json.load(f)

f = open('cities_points.geojson')
point_city = json.load(f)



# new_json = point_city["features"]
# # update_json = json.dump(new_json[0])







# get the lat and longs of cities
# append them in their own list
x = []
y = []
points = []
points_list = []
for r in cities_points:
     x.append(r['latitude'])
     y.append(r['longitude'])
     points_list.append([r['longitude'],r['latitude']])
     


 # use the list to be placed into geopandas to get points 
s = gpd.GeoSeries(gpd.points_from_xy(y, x))
# set the crs to  states crs
s.crs = "epsg:4326"


states_df = gpd.read_file('states.geojson')
points_df = gpd.read_file('cities_points.geojson')

feature_list = []                                                                             
states_json = eval(states_df.to_json(drop_id=True))
points_json = eval(points_df.to_json(drop_id=True))

for f in states_json["features"]:
  feature_list.append(f)

new_json = points_json["features"]





feature_list.append(new_json)


#https://stackoverflow.com/questions/63955752/topologicalerror-the-operation-geosintersection-r-could-not-be-performed
# buffer is to make the mutiploygon be able to work
# solution taken from stackoverflow

states_poly = gpd.GeoSeries(states_df['geometry'].buffer(0))
# print(states_poly[0])
# print(cities_points[62])
# holds the points and index 
results = {}
for i in range(len(states_poly)):
  points_bool =  s.within(states_poly[i])
  results[i] =[]
  for j in range(len(points_bool)):
    if points_bool[j]:
      results[i].append(j)








states_citiesP = []
for k,v in results.items():
    # print(f"{k}:{len(v)}")
    states_citiesP.append([k,v])
    
for k,v in states_citiesP:
  # gives each feature a points to see how many points are it it
  feature_list[k]["properties"]["points"] = [len(v)]
  
  #making sure the points are witin the feature list 
  # print(feature_list[k]["properties"]["points"])

 # coloring the states by the points in each states makes it easer then population 

  if feature_list[k]["properties"]["points"] <= [10]:
            feature_list[k]["properties"]["fill"]= '#7EC8E3'
            feature_list[k]["properties"]["fill-opacity"]=1
            feature_list[k]["properties"]["stroke-width"] = 1
            # print("under 10 points")
  elif feature_list[k]["properties"]["points"] > [10] and feature_list[k]["properties"]["points"] <= [25] :
            feature_list[k]["properties"]["fill"]= '#0000FF'
            feature_list[k]["properties"]["fill-opacity"]= 1
            feature_list[k]["properties"]["stroke-width"] = 1
            # print("between 10 = 25")
  elif feature_list[k]["properties"]["points"] > [25] and feature_list[k]["properties"]["points"] <= [45] :
            feature_list[k]["properties"]["fill"]= '#000C66'
            feature_list[k]["properties"]["fill-opacity"]=1
            feature_list[k]["properties"]["stroke-width"] = 1
            # print("between 25 =  45")
  elif feature_list[k]["properties"]["points"] >= [50]:
            feature_list[k]["properties"]["fill"]= '#050A30'
            feature_list[k]["properties"]["fill-opacity"]=1
            feature_list[k]["properties"]["stroke-width"] = 1
            # print("50 and more ")


# for p in point_city:
#     point_city[p]["properties"]["marker-size"] = "small"

# feature_list.append(point_city)





FeatureCollection = {}
FeatureCollection["type"] = "FeatureCollection"
FeatureCollection["features"] = feature_list

with open("heat_map.geojson", 'w') as f:
  json.dump(FeatureCollection,f,indent=2)




# def pointToPolygon(coords):
#     lon = coords[0]
#     lat = coords[1]

#     #geojson coords in long lat   
#     west = lat - 0.1
#     east = lat + 0.1
#     north = lon + 0.1
#     south = lon - 0.1
#     return [[lon, lat],[south,lat],[south,west],[lon,west],[lon,lat]]



                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     
